<a href="https://colab.research.google.com/github/UrologyUnbound/SIOP_ML_2024_Discord/blob/main/colabs/Empathy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Empathy Responses Notebook
This notebook is designed to tackle the challenge of predicting whether empathy was demonstrated in simulated workplace responses.  

# Challenge Description

Job candidates were asked to provide empathetic responses to a workplace situation. Our task is to classify whether empathy was demonstrated or not in each simulated response.

In [ ]:
!pip install pandas langchain langchain_openai

In [5]:
import pandas as pd
import os
from langchain import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from google.colab import userdata

In [4]:
empathy_train_data = pd.read_csv("https://raw.githubusercontent.com/UrologyUnbound/SIOP_ML_2024_Discord/main/data/train/empathy_train.csv")
empathy_dev_data = pd.read_csv("https://raw.githubusercontent.com/UrologyUnbound/SIOP_ML_2024_Discord/main/data/dev/empathy_val_public.csv")

In [6]:
# Manually add env key to the `api_key` argument
llm = ChatOpenAI(api_key= userdata.get('OPENAI_API_KEY'), temperature=0.3)

In [7]:
instructions = """
Your task is to classify the provided text as either "Empathetic" or "Non-empathetic".
Empathetic Responses involve understanding, supportiveness, and active engagement. Understanding is demonstrated by showing comprehension
of the individual's feelings and perspective. Supportiveness entails offering genuine support, guidance, or constructive feedback while
respecting the individual's contributions and feelings. Active engagement is displayed through asking questions or suggesting actions that
actively engage with the individual's situation. Non-Empathetic Responses lack empathetic qualities. They may lack personalization, offering
generic advice or feedback without addressing the individual's specific feelings or situation. Dismissiveness occurs when the individual's
feelings, concerns, or contributions are downplayed or ignored. Superficiality refers to appearing empathetic on the surface but lacking depth
in understanding or supporting the individual's actual needs.
"""

In [8]:
def extract_examples_empathy(dataset_row):
    # Extract response and empathy from each row of the dataset
    response = dataset_row["text"]
    empathy = dataset_row["empathy"]

    return [{"response": response, "empathy": empathy}]

def create_examples_empathy(dataset):
    # Extract examples from the first three rows of the dataset
    examples = []
    for i in range(3):
        examples.extend(extract_examples_empathy(dataset.loc[i]))

    return examples

def create_example_prompt_empathy():
    # Create a formatter for the examples
    example_prompt = PromptTemplate(
        input_variables=["response", "empathy"],
        template="Response: {response}\nEmpathy: {empathy}"
    )

    return example_prompt

def create_template_empathy(dataset):
    # Generate a few shot prompt template

    examples = create_examples_empathy(dataset)
    template = FewShotPromptTemplate(
        prefix = instructions,
        examples=examples,
        example_prompt=create_example_prompt_empathy(),
        suffix="Response: {input}",
        input_variables=["input"],
    )

    return template

In [9]:
example_prompt = create_template_empathy(empathy_train_data).format(input=empathy_dev_data["text"][0])
example_prompt

'\nYour task is to classify the provided text as either "Empathetic" or "Non-empathetic". \nEmpathetic Responses involve understanding, supportiveness, and active engagement. Understanding is demonstrated by showing comprehension \nof the individual\'s feelings and perspective. Supportiveness entails offering genuine support, guidance, or constructive feedback while \nrespecting the individual\'s contributions and feelings. Active engagement is displayed through asking questions or suggesting actions that \nactively engage with the individual\'s situation. Non-Empathetic Responses lack empathetic qualities. They may lack personalization, offering \ngeneric advice or feedback without addressing the individual\'s specific feelings or situation. Dismissiveness occurs when the individual\'s \nfeelings, concerns, or contributions are downplayed or ignored. Superficiality refers to appearing empathetic on the surface but lacking depth \nin understanding or supporting the individual\'s actual

In [10]:
llm.invoke(example_prompt)

AIMessage(content='Empathy: 1', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 949, 'total_tokens': 954}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})